In [1]:
import pandas as pd
import numpy as np


### simple Logistic Regression

In [2]:
X=pd.read_csv('train.csv')
TEST=pd.read_csv('test.csv')
print('surviver %:',np.mean(X.Survived))

surviver %: 0.3838383838383838


In [3]:

X=X.drop(['PassengerId','Name','Cabin','Ticket'],axis=1)
TEST=TEST.drop(['PassengerId','Name','Cabin','Ticket'],axis=1)

In [4]:
# replace strings with numbers
X['Sex']=np.where(X.Sex=='male',1,0)
X.Embarked=X.Embarked.replace({'Q':0,'S':1,'C':2})
TEST['Sex']=np.where(TEST.Sex=='male',1,0)
TEST.Embarked=TEST.Embarked.replace({'Q':0,'S':1,'C':2})

In [5]:
# drop columns and fill NaN values
mean_age=X.Age.mean()
X=X.fillna(value={'Age':mean_age,'Embarked':0})
mean_test_age=TEST.Age.mean()
TEST=TEST.fillna(value={'Age':mean_age})
Y=X['Survived'].values
X=X.drop(['Survived'],axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,1.0
1,1,0,38.0,1,0,71.2833,2.0
2,3,0,26.0,0,0,7.9250,1.0
3,1,0,35.0,1,0,53.1000,1.0
4,3,1,35.0,0,0,8.0500,1.0


In [6]:
TEST.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null int64
dtypes: float64(2), int64(5)
memory usage: 22.9 KB


In [7]:
mean_age


29.69911764705882

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [9]:
from sklearn.model_selection import GridSearchCV
def classify(model,X,Y,params,cv_folds):
    #
    gd=GridSearchCV(model,params,cv=cv_folds)
    gd.fit(X,Y)
    print("best_parametes: ",gd.best_params_)
    return gd.best_estimator_
    

In [10]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
params={'penalty': ['l1'],'C':[2/(2**i) for i in range(10)],
       'solver':['liblinear'],'max_iter':[200]}
best=classify(lr,X_train,Y_train,params,5)


best_parametes:  {'C': 0.5, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}


In [11]:
from sklearn.metrics import accuracy_score
print('accuracy_score: ', accuracy_score(best.predict(X_test),Y_test))

accuracy_score:  0.7985074626865671


In [12]:
T=pd.read_csv('test.csv')
result=T[['PassengerId']]

TEST=TEST.fillna(value={'Fare':TEST.Fare.mean()})
result.loc[:,'Survived']=pd.Series(best.predict(TEST))

/home/miriam/Escritorio/data/analisis_propiedades/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/miriam/Escritorio/data/analisis_propiedades/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
result.to_csv('prediction.csv',index=False)

### Decision Tree

In [14]:
from sklearn import tree

In [15]:
dt=tree.DecisionTreeClassifier(random_state=0)
params={'max_depth':[i for i in range(1,10)],'max_features':[i for i in range(1,8)]}
best=classify(dt,X_train,Y_train,params,5)
print('accuracy_score: ', accuracy_score(best.predict(X_test),Y_test))

best_parametes:  {'max_depth': 4, 'max_features': 7}
accuracy_score:  0.8208955223880597


In [16]:
result2=T[['PassengerId']]
result2.loc[:,'Survived']=pd.Series(best.predict(TEST))

/home/miriam/Escritorio/data/analisis_propiedades/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/miriam/Escritorio/data/analisis_propiedades/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
result2.to_csv('prediction2.csv',index=False)